In [195]:
import pandas as pd
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
import numpy as np
import psycopg2
from psycopg2.extras import NamedTupleCursor
import pandas.io.sql as sqlio
from efficient_apriori import apriori

In [196]:
conn = psycopg2.connect(dbname="postgres", user="admin_film", password="111", host="127.0.0.1", port="5432")
print("Подключение установлено")

Подключение установлено


In [197]:
sql = """with actor_name as(
select film_id,first_name,last_name 
from actor
join film_actor using(actor_id)
)

select title,first_name,last_name,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features
from film 
join actor_name using(film_id);"""
film = pd.read_sql_query(sql, conn)
film.head()

,title,first_name,last_name,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,ACADEMY DINOSAUR,PENELOPE,GUINESS,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"[Deleted Scenes, Behind the Scenes]"
1,ANACONDA CONFESSIONS,PENELOPE,GUINESS,A Lacklusture Display of a Dentist And a Denti...,2006,3,0.99,92,9.99,R,"[Trailers, Deleted Scenes]"
2,ANGELS LIFE,PENELOPE,GUINESS,A Thoughtful Display of a Woman And a Astronau...,2006,3,2.99,74,15.99,G,[Trailers]
3,BULWORTH COMMANDMENTS,PENELOPE,GUINESS,A Amazing Display of a Mad Cow And a Pioneer w...,2006,4,2.99,61,14.99,G,[Trailers]
4,CHEAPER CLYDE,PENELOPE,GUINESS,A Emotional Character Study of a Pioneer And a...,2006,6,0.99,87,23.99,G,"[Trailers, Commentaries, Behind the Scenes]"


In [198]:
df_categorial = film[['title','description','rating','last_name']]

In [199]:
columns = list(df_categorial.columns)

transactions = []
for i in range(len(df_categorial)):
    values=tuple(list(t.strip() for t in df_categorial.iloc[i,:].values if (t is not None) and (not isinstance(t,float))))
    transactions.append(values)

In [206]:
items,rules = apriori(transactions,min_support=0.003,min_confidence=0.25)

In [207]:
rules

[{AKROYD} -> {PG-13},
 {ALLEN} -> {PG-13},
 {BERRY} -> {PG-13},
 {DAVIS} -> {NC-17},
 {DEGENERES} -> {NC-17},
 {TEMPLE} -> {G},
 {WILLIS} -> {G},
 {GARLAND} -> {PG-13},
 {GOODING} -> {PG-13},
 {HARRIS} -> {NC-17},
 {HOPKINS} -> {NC-17},
 {JOHANSSON} -> {NC-17},
 {MCQUEEN} -> {PG},
 {MOSTEL} -> {PG-13},
 {TORN} -> {NC-17},
 {PECK} -> {PG-13},
 {SILVERSTONE} -> {PG},
 {ZELLWEGER} -> {PG}]

In [219]:
sql = """select last_name,title,description,release_year,rating,amount
from customer as c
join payment as p using(customer_id)
join rental as r using(rental_id)
join inventory as i using(inventory_id)
join film as f using(film_id);"""
film_buy = pd.read_sql_query(sql, conn)
film_buy.head()

,last_name,title,description,release_year,rating,amount
0,WALTERS,PACKER MADIGAN,A Epic Display of a Sumo Wrestler And a Forens...,2006,PG-13,0.99
1,JENNINGS,CHARIOTS CONSPIRACY,A Unbelieveable Epistle of a Robot And a Husba...,2006,R,2.99
2,RHODES,CRAFT OUTFIELD,A Lacklusture Display of a Explorer And a Hunt...,2006,NC-17,0.99
3,BILLINGSLEY,CRUSADE HONEY,A Fast-Paced Reflection of a Explorer And a Bu...,2006,R,2.99
4,QUINTANILLA,ROMAN PUNK,A Thoughtful Panorama of a Mad Cow And a Stude...,2006,NC-17,0.99


In [220]:
df_categorial = film_buy.select_dtypes(include=['object'])

In [221]:
columns = list(df_categorial.columns)

transactions = []
for i in range(len(df_categorial)):
    values=tuple(list(t.strip() for t in df_categorial.iloc[i,:].values if (t is not None) and (not isinstance(t,float))))
    transactions.append(values)

In [244]:
items,rules = apriori(transactions,min_support=0.002,min_confidence=0.4)

In [245]:
rules

[{BUCKET BROTHERHOOD} -> {A Amazing Display of a Girl And a Womanizer who must Succumb a Lumberjack in A Baloon Factory},
 {A Amazing Display of a Girl And a Womanizer who must Succumb a Lumberjack in A Baloon Factory} -> {BUCKET BROTHERHOOD},
 {A Amazing Display of a Girl And a Womanizer who must Succumb a Lumberjack in A Baloon Factory} -> {PG},
 {A Awe-Inspiring Character Study of a Robot And a Sumo Wrestler who must Discover a Womanizer in A Shark Tank} -> {PG-13},
 {ROCKETEER MOTHER} -> {A Awe-Inspiring Character Study of a Robot And a Sumo Wrestler who must Discover a Womanizer in A Shark Tank},
 {A Awe-Inspiring Character Study of a Robot And a Sumo Wrestler who must Discover a Womanizer in A Shark Tank} -> {ROCKETEER MOTHER},
 {BUCKET BROTHERHOOD} -> {PG},
 {ROCKETEER MOTHER} -> {PG-13},
 {BUCKET BROTHERHOOD, PG} -> {A Amazing Display of a Girl And a Womanizer who must Succumb a Lumberjack in A Baloon Factory},
 {A Amazing Display of a Girl And a Womanizer who must Succumb a Lu

In [239]:
film_buy[film_buy['description'] == 'A Amazing Display of a Girl And a Womanizer who must Succumb a Lumberjack in A Baloon Factory']

,last_name,title,description,release_year,rating,amount
862,WATKINS,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
1074,BANDA,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
2612,MCADAMS,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,5.99
2782,MURPHY,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
3595,SANDERS,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
3903,BOUDREAU,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
3930,ISOM,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
4213,LOMBARDI,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
5214,BARBEE,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99
6222,MILLER,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,PG,4.99


In [246]:
### жанр проверь 
sql = """with genre as(
select name,film_id from category
join film_category using(category_id)
)

select last_name,title,description,release_year,rating,name,amount
from customer as c
join payment as p using(customer_id)
join rental as r using(rental_id)
join inventory as i using(inventory_id)
join film as f using(film_id)
join genre using(film_id);"""
film_genre = pd.read_sql_query(sql, conn)
film_genre.head()

,last_name,title,description,release_year,rating,name,amount
0,WALTERS,PACKER MADIGAN,A Epic Display of a Sumo Wrestler And a Forens...,2006,PG-13,Animation,0.99
1,JENNINGS,CHARIOTS CONSPIRACY,A Unbelieveable Epistle of a Robot And a Husba...,2006,R,Sci-Fi,2.99
2,RHODES,CRAFT OUTFIELD,A Lacklusture Display of a Explorer And a Hunt...,2006,NC-17,Drama,0.99
3,BILLINGSLEY,CRUSADE HONEY,A Fast-Paced Reflection of a Explorer And a Bu...,2006,R,Sports,2.99
4,QUINTANILLA,ROMAN PUNK,A Thoughtful Panorama of a Mad Cow And a Stude...,2006,NC-17,Music,0.99


In [248]:
df_categorial = film_genre[['last_name','rating','name']]

In [250]:
columns = list(df_categorial.columns)

transactions = []
for i in range(len(df_categorial)):
    values=tuple(list(t.strip() for t in df_categorial.iloc[i,:].values if (t is not None) and (not isinstance(t,float))))
    transactions.append(values)

In [285]:
items,rules = apriori(transactions,min_support=0.004,min_confidence=0.28)

In [286]:
rules

[{Animation} -> {PG-13},
 {Comedy} -> {PG},
 {Documentary} -> {PG},
 {Drama} -> {PG-13},
 {Family} -> {PG},
 {Foreign} -> {PG-13},
 {Games} -> {NC-17},
 {Music} -> {NC-17},
 {New} -> {NC-17}]

### Ассоциативный анализ показывает взаимосвязи между категориальными признаками. Эти взаимосвязи нужны для того,чтобы обратить особое внимание при визуализации,ответах на вопросы,построение гипотез на пары.
### Результат анализа зависимостей названия,описания,рейтинга и фамилии актера: метод априори связал фамилии актеров с рейтингом сериала.Значит актеру с определенной фамилией соответствует определенный жанр,это можно использовать при рекомендациях сериалов пользователям и при предсказывании рейтинга сериала.
### Результат анализа зависимостей между фамилией клиента,рейтингом,названием,описанием: показал зависимость между названием,описанием и рейтингом сериала,что было ожидаемо,зависимости с фамилией клиента обнаружено не было.
### Результат анализа зависимостей между рейтингом,жанром,фамилией клиента: метод показал зависимость между жанрами и рейтингом,что означает,что чаще всего сериал с определенным жанром имеет определенный рейтинг, это поможет настроить рекомендательную и предсказательную систему .